In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import os
from config import destination_zip, col_types

In [ ]:
# read csv in zip pandas automatically selects the csv residing within the zip
accidents_df = pd.read_csv(destination_zip, 
                           usecols=col_types.keys(), 
                           dtype=col_types, 
                           parse_dates=['Start_Time'],
                           infer_datetime_format=True)

accidents_df.head()

In [ ]:
# Chris's code starts here

In [ ]:
#Isolate Start Time and sort by day of week
accidents_df["Start_Time"].dt.dayofweek.value_counts()

In [ ]:
#Create DataFrame to store number of accidents per day of week
week_days_df = pd.DataFrame(accidents_df['Start_Time'].dt.dayofweek.value_counts())
week_days_df.sort_index(inplace=True)
week_days_df.set_index(pd.Index(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']), inplace=True)
week_days_df

In [ ]:
plotdata = week_days_df.plot.bar(rot="vertical", title="Number of Accidents by Day of Week", legend=False)
plotdata.set_ylabel("Number of Accidents")

In [ ]:
# Chris's code ends here

In [ ]:
# Colleen's code starts here

In [ ]:
# Colleen's code ends here

In [ ]:
# Ken's code starts here

In [ ]:
# Ken's code ends here

In [ ]:
# Eric's code starts here

In [ ]:
# Eric's code ends here